In [ ]:
folder_path = "/content/drive/MyDrive/PhD/OOD_VQA/Exps"
%cd $folder_path

/content/drive/MyDrive/PhD/OOD_VQA/Exps


In [ ]:
!pwd

/content/drive/MyDrive/PhD/OOD_VQA/Exps


# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import math
from utils import *

# Load results

In [ ]:
image_names = ["Gaussian_5_2", "Gaussian_5_3", "Gaussian_5_4", "Gaussian_5_5",
               "white_house_brown_cat", "white_house_brown_cow", "white_house_brown_dog", "white_house_brown_horse",
               "white_house_green_cat", "white_house_green_unknown",
               "white_house_pink_cow", "white_house_pink_unknown",
               "white_house_purple_dog", "white_house_purple_horse",
               "white_house_red_duck", "white_house_red_unknown",
               "white_house_yellow_unknown"]

criteria = ["image_type", "used_language", "specificity", "question_length", "complexity", "image_relatedness"]
fixed_cols = set(["question", "answer_gt_1", "answer_gt_2"] + criteria)

In [ ]:
model_types = ["VQA", "IC"]
model_names = ["GiT_base", "GiT_large", "Blip_base", "Blip_large", "Vilt"]

models_dict = dict()

for model_type in model_types:
  t = dict()
  for model_name in model_names:
    t[model_name] = None
  models_dict[model_type] = t.copy()

models_dict

{'VQA': {'GiT_base': None,
  'GiT_large': None,
  'Blip_base': None,
  'Blip_large': None,
  'Vilt': None},
 'IC': {'GiT_base': None,
  'GiT_large': None,
  'Blip_base': None,
  'Blip_large': None,
  'Vilt': None}}

## Load answers

In [ ]:
# all_results_df = pd.DataFrame()

# for image_name in image_names:
#   image_results_df = pd.DataFrame()

#   for model_type in model_types:

#     for model_name in model_names:

#       results_path = f"./test_results/{model_type}/{model_name}/"
#       file_name = f"{model_type}_{model_name}_{image_name}"
#       results_path = results_path + file_name + ".xlsx"
#       if not os.path.isfile(results_path): f"!!! Warning: No file !!!\n{file_name} sheet is not available!"
#       df = pd.read_excel(results_path, sheet_name='answers')

#       image_results_df[f"{model_type}_{model_name}_question"] = df["question"]
#       image_results_df[f"{model_type}_{model_name}_correct"] = df["correct"]
#       image_results_df[f"{model_type}_{model_name}_correct"]



In [ ]:
all_results_df = pd.DataFrame()

model_types = ['VQA']
for model_type in model_types:
  for image_name in image_names:
    for model_name in model_names:
      results_path = f"./test_results/{model_type}/{model_name}/"
      file_name = f"{model_type}_{model_name}_{image_name}"
      results_path = results_path + file_name + ".xlsx"
      if not os.path.isfile(results_path): f"!!! Warning: No file !!!\n{file_name} sheet is not available!"
      df = pd.read_excel(results_path, sheet_name='answers', usecols="B:F")
      columns = [f"{model_name}_{c}" for c in df.columns]
      df.columns = columns.copy()
      image_name_col = [image_name for _ in range(len(df))]
      df.insert(loc=0, column='image_name', value=image_name_col)
      all_results_df = all_results_df.append(df, ignore_index = True)
      # model_answers_dict[model_type][model_name][image_name] = df.copy()
all_results_df.head()

In [ ]:
model_type = "VQA"

all_results_df = pd.DataFrame()

for image_name in image_names:
  image_df = pd.DataFrame()
  new_image = True
  for model_name in model_names:
    results_path = f"./test_results/{model_type}/{model_name}/"
    file_name = f"{model_type}_{model_name}_{image_name}"
    results_path = results_path + file_name + ".xlsx"
    if not os.path.isfile(results_path): f"!!! Warning: No file !!!\n{file_name} sheet is not available!"
    usecols="B:M" if new_image else "K:M"
    df = pd.read_excel(results_path, sheet_name='answers', usecols=usecols)
    for c in df.columns:
      # if ("question" in c) or ("gt" in c): new_col_name = c
      if c in fixed_cols: new_col_name = c
      else: new_col_name = f"{model_name}_{c}"
      image_df[new_col_name] = df[c]
    new_image = False
    # columns = [f"{model_name}_{c}" for c in df.columns]
    # df.columns = columns.copy()
    # image_df.concat([image_df, df], axis=1)
  image_name_col = [image_name for _ in range(len(image_df))]
  image_df.insert(loc=0, column='image_name', value=image_name_col)
  all_results_df = all_results_df.append(image_df, ignore_index = True)

all_results_df.to_excel(f"./test_results/{model_type}/{model_type}_test_answers.xlsx", sheet_name=f"{model_type}_answers")

<ipython-input-63-91005504e1cd>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results_df = all_results_df.append(image_df, ignore_index = True)
<ipython-input-63-91005504e1cd>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results_df = all_results_df.append(image_df, ignore_index = True)
<ipython-input-63-91005504e1cd>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results_df = all_results_df.append(image_df, ignore_index = True)
<ipython-input-63-91005504e1cd>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results_df = all_results_df.append(image_df, ignore_index = True)
<ipython-input-63-91005504e1cd>:25: 

# Load logits

In [ ]:
model_type = "VQA"

logits_dict = dict()

for model_name in model_names:
  model_dict = dict()
  for image_name in image_names:
    results_path = f"./test_results/{model_type}/{model_name}/"
    file_name = f"{model_type}_{model_name}_{image_name}"
    results_path = results_path + file_name + ".npy"
    if not os.path.isfile(results_path): f"!!! Warning: No file !!!\n{file_name} file is not available!"
    l = np.load(results_path, allow_pickle=True)
    model_dict[image_name] = l.copy()
  logits_dict[model_name] = model_dict.copy()

In [ ]:
np.save(f"./test_results/{model_type}/{model_type}_test_logits.npy", logits_dict)

In [ ]:
# Lo = np.load(f"./test_results/{model_type}/{model_type}_test_logits.npy", allow_pickle=True)
# D = Lo.item()

<class 'numpy.ndarray'>
